## The Fourier Motzkin Elimination Algorithm
This algorithm computes a projection of a polyhedron by eliminating a given axis.  
The result is a polyhedron given by $P(D,d) = {Proj}_k(P(A,b))=\{x\in K^n:Dx\leq d, x_k=0\}$.  
It can be run on three test-files containing polyhedrons of different sizes.

In [20]:
# Imports etc.
import numpy as np

In [21]:
# Parameters
input_path = "test1.txt"
elimination_index = 1
output_path = "test1_results.txt"

In [22]:
# Read-In polyhedron data from .txt file
with open(input_path, 'r') as file:
    P = np.matrix([[int(num) for num in line.split(' ')] for line in file])
b = P[:, 0]
A = P[:, 1:]
print("Read-in polyhedron.")
print(f"Shape of b is {b.shape}")
print(f"Shape of A is {A.shape}")

Read-in polyhedron.
Shape of b is (10, 1)
Shape of A is (10, 5)


In [25]:
# Main algorithm
def fme(k):
    k = k - 1
    """ Fourier Motzkin Elemination
    
    :param: k: The index (axis) to be eliminated.
    :return: Projected polyhedron
    """
    # We first partition our set of indices of A
    C_k_0 = np.where(A[:,k] == 0)[0]
    C_k_minus = np.where(A[:,k] < 0)[0]
    C_k_plus = np.where(A[:,k] > 0)[0]
    # Get R and Bijection
    R = ...
    # Iterate

[[ 1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]
[1 2 3 4 6 7 8 9]
